# ETL 00: Reading and Pre-processing NFT data (raw --> stage --> NDS)
Reading the transaction file(s), and preprocess them for smaller tables. As the input file is huge, the following method is used:
 * ETL raw --> stage: split the big input table(s) into smaller, redundant pieces by topic (keeping columns only which needed for a table) and by rows (reading by million rows)
   * The file is loaded in pieces (nrows, skiprows)
   * Preprocessing is made on the loaded partial file (for all future tables: e.g. save all token-collection pairs)
 * ETL stage --> nds: The next layer (agregator) picks up these partial tables and adding them to the database (by topic, handling the duplications).
 * Cleaner/Archiver functions are also needed

</br>
A pyspark version will be also created, but this can be run even in a home PC (alternative solutions like dask will be explored as well).

In [2]:
# loading ETL related libraries
import pandas as pd
import numpy as np

# core libraries
from datetime import datetime, timedelta
import imp

# OS related
from os import listdir, makedirs, remove, path

# parallel programming related
from multiprocessing import Pool
import subprocess

# visualization related
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# settings
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

# src loader functions, pathes, config values
import src.config as cf
import src.ETL_00_rawToStage as ers
import src.ETL_01_stageToNDS as esn
import src.ETL_02_analyticsDM as edm

In [64]:
ers = imp.reload(ers)
esn = imp.reload(esn)
edm = imp.reload(edm)
cf = imp.reload(cf)

## Raw to Stage
Loading the input file (and separate them to smaller tables). Inputs (stored in config file): 
 * input_folder: the folder where the file(s) are stored
 * filename: the name of the file which should be read (if empty, reading all files from the folder)
 * batchsize: max read row number
 * njobs: # of processors for parallel computing
 * output_main_folder: the main folder where the output files are stored (output name convention is "topic_sourcename_startrow_endrow.pickle")
 * category_list: the list of considered categories (if empty, all categories are used)

In [21]:
# example run - only for a part of the input file (only for debugging/testing)

_example_input = {'path': './data/00_raw/Data_API.csv', 'start_row': 4000000, 'to_read': 500000, 
                  'out_path_base': './data/01_stage/', 'categories': ['Art']}

ers.ReadOneChunk(_example_input)
pdf_test_trx = pd.read_pickle('./data/01_stage/trx/Data_API_4000000_500000.pickle')
pdf_test_token = pd.read_pickle('./data/01_stage/token/Data_API_4000000_500000.pickle')

print(pdf_test_trx.shape[0], pdf_test_token.shape[0])

233836 204848


In [22]:
# calling the parallel runner for all
ers.ReadAllFiles_parallel(input_folder=cf.PATH_00RAW_API_input_folder, input_file=cf.PATH_00RAW_API_input_filename, 
                          batchsize=cf.PARAM_batchsize, category_list=cf.PARAM_categorylist, 
                          output_folder=cf.PATH_01STAGE_tmp_main, cln_flg=cf.PARAM_raw_cleanup, 
                          njobs=2, _verbose=True)

[Wed 13:30:09] Data preprocessing started running (raw to stage)...
[Wed 13:30:09] Planning the parallel execution...
[Wed 13:30:33] Starting parallel execution...
[Wed 13:34:00] Finished running...


In [24]:
# testing some files (from the read ones)
# basic centent test, and some ID tests

pdf_test_trx = pd.read_pickle('./data/01_stage/trx/Data_API_4000000_500000.pickle')
pdf_test_token = pd.read_pickle('./data/01_stage/token/Data_API_4000000_500000.pickle')

print(pdf_test_trx.shape[0], pdf_test_token.shape[0])
print(pdf_test_token.tail(42000).head(2).iloc[0].token_URL_latest)
# print(pdf_test_token.token_crypto.value_counts())
print("----------------------")
print(pdf_test_trx.trx_value_usd.isnull().sum(), pdf_test_trx.trx_value_crypto.isnull().sum())

233836 204848
https://lh3.googleusercontent.com/gocq_BBBapbj6JiAdjoy0i7vu6iV7nOvhtiW0DXxKvRlwe7AgYwABVzHJ-JwaHvapkr_kJEiXPt3pVm_5aj2s8rUVKHOsHx7TY15qQ=s128
----------------------
75 0


## Stage to NDS
Processing the partial tables and create NDS tables. These will be the base of the DM analysis tables. 2 tables will be made:
 * a token master table: which contains a unified token table - separated file, by category
 * a transaction master table: which is splitted by yearmonth and category

In [40]:
# testing the transaction writer
# esn._addTrxToNDS(path_partial_file=cf.PATH_01STAGE_tmp_main + 'trx/Data_API_0_500000.pickle', path_output_main=cf.PATH_02NDS_main)
# esn._addTrxToNDS(path_partial_file=cf.PATH_01STAGE_tmp_main + 'trx/Data_API_500000_500000.pickle', path_output_main=cf.PATH_02NDS_main)
# pdf_transactions_chk = pd.read_pickle(cf.PATH_02NDS_main + 'transactions/art/trx_201712.pickle')
# print(pdf_transactions_chk.shape[0])
# pdf_transactions_chk.head(2)

In [41]:
# testing the token writer
# esn._addTokensToNDS(path_partial_file=cf.PATH_01STAGE_tmp_main + 'token/Data_API_0_500000.pickle', path_output_main=cf.PATH_02NDS_main)
# esn._addTokensToNDS(path_partial_file=cf.PATH_01STAGE_tmp_main + 'token/Data_API_500000_500000.pickle', path_output_main=cf.PATH_02NDS_main)
# pdf_tokens_chk = pd.read_pickle(cf.PATH_02NDS_main + 'tokens/token_master_art.pickle')
# print(pdf_tokens_chk.shape[0])
# print(pdf_tokens_chk.token_ID.value_counts().max())
# pdf_tokens_chk.head(2)

In [61]:
esn.stageToNDS(in_path_stage=cf.PATH_01STAGE_tmp_main, out_path_NDS=cf.PATH_02NDS_main, _verbose=True)

[Wed 15:52:22] Transaction data loader started running (stage to NDS)...
[Wed 15:52:37] Token master loader started running (stage to NDS)...
[Wed 15:53:33] Finished running...


In [2]:
pdf_nft_master = pd.read_pickle(cf.PATH_02NDS_main + '/tokens/token_master_art.pickle')
pdf_nft_master.head(2)

,collection_ID,token_ID,token_URL_latest,token_permanent_link,token_mint_crypto,token_mint_price_USD,token_mint_price_crypto,token_mint_address_est,token_mint_date_est,token_latest_sales_crypto,token_latest_sales_date,token_latest_price_USD,token_latest_price_crypto,token_collection_nm,token_name,token_description,token_category,token_ID_error
0,Cryptokitties,Cryptokitties_1268119,https://lh3.googleusercontent.com/SWIgsdqj2cUc...,https://opensea.io/assets/0x06012c8cf97bead5de...,ETH,0.321473,0.001980,0xb367b96bd9af396dc5281cfdcd9e9571f670832f,2019-04-27 17:45:40,ETH,2021-04-12 17:25:55,19.363050,0.0090,Cryptokitties,I Miss You,"Guten tag. I'm I Miss You, fairy princess of C...",Art,0
1,Cryptokitties,Cryptokitties_1434540,https://lh3.googleusercontent.com/7ILL0U1wFba7...,https://opensea.io/assets/0x06012c8cf97bead5de...,ETH,0.402917,0.001487,0xfc624f8f58db41bdb95aedee1de3c1cf047105f1,2019-03-10 03:11:34,WETH,2021-04-10 11:55:04,56.558319,0.0273,Cryptokitties,Lieutenant Bippitynose,Oh heyyy. I'm Lieutenant Bippitynose. Are you ...,Art,0


## Run Data Cleansing
Clean the transactions from duplications (mark them, the DM loaders will not read the duplicated records).

In [16]:
edm = imp.reload(edm)
esn.log("Started Running Cleaner...")
edm.TrxDuplicationMarker(cf.PATH_02NDS_main + 'transactions/art/')
esn.log("Finished Running Cleaner...")

[Sun 12:37:10] Started Running Cleaner...
[Sun 12:38:13] Finished Running Cleaner...


In [17]:
# checking the results with an example
pdf_chk = edm._readTrx_betweenMonats(cf.PATH_02NDS_main + 'transactions/art/', load_max_intmonat=201803)
pdf_chk[pdf_chk.token_ID == 'Cryptokitties_597444']

,Market_ID,collection_ID,token_ID,token_category,trx_time_sec,trx_seller_address,trx_buyer_address,trx_crypto,trx_value_crypto,trx_value_usd,trx_duplication_flg
358160,Cryptokitties,Cryptokitties,Cryptokitties_597444,Art,2018-03-09 01:42:53,0x77cd1ae1559b5357392a8ec4870cdd92a827815e,0xe97c9fd7eb06fae12b69661a1d6379cefad16268,ETH,0.069989,49.918648,0
358161,OpenSea,Cryptokitties,Cryptokitties_597444,Art,2018-03-10 02:01:23,0x77cd1ae1559b5357392a8ec4870cdd92a827815e,0xe97c9fd7eb06fae12b69661a1d6379cefad16268,ETH,0.069989,49.918648,1


In [45]:
# pdf_chk.trx_time_sec.dt.date

## Processing DM tables
Create tables for analysis. It creates 3 layers of tables. 
 * A temporary layer with a few base tables, which makes the DM load faster (e.g. token - user monthly snapshots)
 * DM tables about different topics: contains tables which could be the basics of analysis and simulation, and reports (like traders' monthly positions, collection time serier, etc.)
 * ABT's which can be the base of the models and NFT-trading strategy simulations. These will have 2 main loader modes: the "test_strategy" method creates all tables up to a date, and record (where possible) the transactions/prices from the next x months. This can be used for training models and for trying out strategies. The "scoring" mode creates all tables up to a date (preferably "today").

### Temporary layer
Contains tables which are not used for analysis, but which are the common base of the more complex DM tables. Most of them are not necessary in the pyspark mode.
</br></br>
**Note**: There are tokens which exist in multiple pieces - and also, some transactions sells many of them. These are not really NFTs, and the current data cannot handle them well. We will mark them, and some of them will be excluded from the transactions. Example [here](https://opensea.io/assets/0xd07dc4262bcdbf85190c01c996b4c06a461d2430/37408). Later on, we will download this feature as well (when using own API caller).

#### NFTU - User Table

In [95]:
# tester - part 1: first run

# edm._tempL_NFT_User_pairs_mthly(cf.PATH_02NDS_main + 'transactions/art/', cf.PATH_03DM_01tempL + 'nftu/art/', 201711) # first month
# tmp_chk = pd.read_pickle(cf.PATH_03DM_01tempL + 'art/nftu/nftuser_201711.pickle')
# print(tmp_chk.shape[0], tmp_chk.nftu_hold_flg.value_counts().sum(), 
#       tmp_chk.nftu_mint_flg.sum() / tmp_chk.nftu_sell_cnt.sum(), 
#       tmp_chk.nftu_hold_flg.sum() / tmp_chk.shape[0])
# tmp_chk.head(2)

In [96]:
# tester - part 2: a few appends

# edm._tempL_NFT_User_pairs_mthly(cf.PATH_02NDS_main + 'transactions/art/', cf.PATH_03DM_01tempL + 'nftu/art/', 201712)
# edm._tempL_NFT_User_pairs_mthly(cf.PATH_02NDS_main + 'transactions/art/', cf.PATH_03DM_01tempL + 'nftu/art/', 201801)
# edm._tempL_NFT_User_pairs_mthly(cf.PATH_02NDS_main + 'transactions/art/', cf.PATH_03DM_01tempL + 'nftu/art/', 201802)
# tmp_chk = pd.read_pickle(cf.PATH_03DM_01tempL + 'art/nftu/nftuser_201802.pickle')
# print(tmp_chk.shape[0], tmp_chk.nftu_hold_flg.value_counts().sum(), 
#       tmp_chk.nftu_mint_flg.sum() / tmp_chk.nftu_sell_cnt.sum(), 
#       tmp_chk.nftu_hold_flg.sum() / tmp_chk.shape[0])
# tmp_chk.head(2)

In [18]:
# Run the full layer, and read the latest table
edm.NFTUserPairs_runner(trx_folder_in=cf.PATH_02NDS_main + 'transactions/art/', 
                        nftu_folder_out=cf.PATH_03DM_01tempL + 'nftu/art/', _verbose=True)

[Sun 12:41:18] NFT - User Montly Pair Table started running ...
[Sun 12:41:18] Running 42 month(s) from 201711 to 202104
[Sun 12:42:53] NFT - User Montly Pair Table finished running ...


In [19]:
tmp_chk = pd.read_pickle(cf.PATH_03DM_01tempL + 'nftu/art/nftuser_202103.pickle')
print(tmp_chk.shape[0], tmp_chk.nftu_hold_flg.value_counts().sum(), 
      tmp_chk.nftu_mint_flg.sum() / tmp_chk.nftu_sell_cnt.sum(), 
      tmp_chk.nftu_hold_flg.sum() / tmp_chk.shape[0], 
      tmp_chk.token_ID.nunique())
tmp_chk[tmp_chk.nftu_mint_flg == 1].head(2)

900752 900752 0.4103815948870045 0.8938853313675684 763938


,token_category,collection_ID,token_ID,trader_ID,nftu_hold_flg,nftu_sell_date_latest,nftu_sell_cnt,nftu_sell_amt_usd_sum,nftu_sell_price_usd_latest,nftu_buy_date_latest,nftu_buy_price_usd_latest,nftu_buy_cnt,nftu_buy_amt_usd_sum,nftu_mint_flg,nftu_mint_usd
730627,Art,0xmons-xyz,0xmons-xyz_103,0xf24a0018befb3d7503b46c110f83d927d64e727d,0.0,2021-03-14 10:15:27,2.0,22617.539062,11308.769531,NaT,NaN,0.0,0.0,1.0,11308.769531
730628,Art,0xmons-xyz,0xmons-xyz_112,0xfaa65d761876e51328a58a0f12f3bb193d79b858,0.0,2021-03-04 06:55:54,2.0,20508.511719,10254.255859,NaT,NaN,0.0,0.0,1.0,10254.255859


Tests, problems to solve:
 * **Problem**: In many cases, in one month, 1 trader sells the same token several times. **Solution**: there are NFTs, which has several (up to 100k) tokens, so one can sell several.
 * **Problem**: Too many mints, comparing to unique sells (100k sells, 80k mint - only 20 is normal). **Solution**: probably this is the reality, most of the tokens are only sold once - and the table keeps the trader-NFT pairs, and loading it from the previous month if there were no sales...

In [106]:
# tmp_chk[tmp_chk.nftu_sell_cnt > 0].shape[0]
# tmp_chk.nftu_sell_cnt.value_counts()
# tmp_chk[tmp_chk.nftu_sell_cnt == 5].head(2)

817708

#### NFT KPI base table

In [28]:
# tester - part 1: first run

# edm._tempL_NFT_mthly(pathfolder_trx=cf.PATH_02NDS_main + 'transactions/art/', 
#                      pathfolder_nftuser=cf.PATH_03DM_01tempL + 'nftu/art/', 
#                      pathfolder_nft=cf.PATH_02NDS_main + 'tokens/', 
#                      pathfolder_nftkpi=cf.PATH_03DM_01tempL + 'nftkpi/art/', actmonat=201711, _tokcat='Art')

# tmp_chk = pd.read_pickle(cf.PATH_03DM_01tempL + 'nftkpi/art/nftkpi_201711.pickle')

# print(tmp_chk.shape[0], tmp_chk.nftkpi_mint_trx_cnt_csum.max(), tmp_chk.nftkpi_token_trx_cnt_csum.max(), 
#       tmp_chk.nftkpi_token_parallel_own_max.max(), tmp_chk.nftkpi_coll_trx_cnt_csum.max(), 
#       tmp_chk[tmp_chk.nftkpi_inmonth_trx_flg == 1].shape[0])
# tmp_chk.head(2)

In [29]:
# tester - part 2: next months

# edm._tempL_NFT_mthly(pathfolder_trx=cf.PATH_02NDS_main + 'transactions/art/', 
#                      pathfolder_nftuser=cf.PATH_03DM_01tempL + 'nftu/art/', 
#                      pathfolder_nft=cf.PATH_02NDS_main + 'tokens/', 
#                      pathfolder_nftkpi=cf.PATH_03DM_01tempL + 'nftkpi/art/', actmonat=201712, _tokcat='Art')

# edm._tempL_NFT_mthly(pathfolder_trx=cf.PATH_02NDS_main + 'transactions/art/', 
#                      pathfolder_nftuser=cf.PATH_03DM_01tempL + 'nftu/art/', 
#                      pathfolder_nft=cf.PATH_02NDS_main + 'tokens/', 
#                      pathfolder_nftkpi=cf.PATH_03DM_01tempL + 'nftkpi/art/', actmonat=201801, _tokcat='Art')

# tmp_chk2 = pd.read_pickle(cf.PATH_03DM_01tempL + 'nftkpi/art/nftkpi_201801.pickle')

# print(tmp_chk2.shape[0], tmp_chk2.nftkpi_mint_trx_cnt_csum.max(), tmp_chk2.nftkpi_token_trx_cnt_csum.max(), 
#       tmp_chk2.nftkpi_token_parallel_own_max.max(), tmp_chk2.nftkpi_coll_trx_cnt_csum.max(), 
#       tmp_chk2[tmp_chk2.nftkpi_inmonth_trx_flg == 1].shape[0])
# tmp_chk2.head(2)

In [20]:
# Run the full layer, and read the latest table
edm.NFTKPI_runner(trx_folder=cf.PATH_02NDS_main + 'transactions/', 
                  nftu_folder=cf.PATH_03DM_01tempL + 'nftu/', nftkpi_folder=cf.PATH_03DM_01tempL + 'nftkpi/', 
                  nftfolder=cf.PATH_02NDS_main + 'tokens/', tokencats=['Art'], load_interval_intmonat=[], 
                  _verbose=True)

[Sun 13:00:31] NFT - NFT KPI monthly loader started running ...
[Sun 13:00:31] Art category is running 42 month(s) from 201711 to 202104
[Sun 13:09:09] NFT - User Montly Pair Table finished running ...


In [21]:
tmp_chk = pd.read_pickle(cf.PATH_03DM_01tempL + 'nftkpi/art/nftkpi_202103.pickle')
print(tmp_chk.shape[0], tmp_chk.nftkpi_mint_trx_cnt_csum.max(), tmp_chk.nftkpi_token_trx_cnt_csum.max(), 
      tmp_chk.nftkpi_token_parallel_own_max.max(), tmp_chk.nftkpi_coll_trx_cnt_csum.max(), 
      tmp_chk[tmp_chk.nftkpi_inmonth_trx_flg == 1].shape[0])
tmp_chk.head(2)

794894 41088.0 658.0 474.0 863240.0 97104


,token_category,collection_ID,token_ID,token_mint_address_est,nftkpi_datetime_stamp,nftkpi_token_trx_cnt_csum,nftkpi_token_trx_usd_csum,nftkpi_token_parallel_own_max,trx_value_usd,trx_cnt,token_tenure_days,nftkpi_coll_trx_cnt_csum,nftkpi_coll_trx_usd_csum,nftkpi_mint_trx_cnt_csum,nftkpi_mint_trx_usd_csum,nftkpi_inmonth_trx_flg
0,Art,0xmons-xyz,0xmons-xyz_101,0x6f6101686f5b5c33bb5aac1d9185c9a0ccbdb3cf,2021-02-26 21:11:09,1.0,17781.839844,1.0,17781.839844,1,0,29.0,213976.906250,2.0,25512.320312,0
1,Art,0xmons-xyz,0xmons-xyz_107,0xefe366e42f6e2684ef608581f7b743b0fb8d64d2,2021-02-23 04:56:43,1.0,7933.660156,1.0,7933.660156,1,0,26.0,177101.359375,2.0,16574.281250,0


### DM layer
These tables can be used for exploration and reports directly (or joined), and could be base for the ML/graph models. The following DM tables are made:
 * NFT - Trader graph base table
 * Transactional base table
 * Trader's KPI (by time)
 * Minter's KPI (by time)
 * Collection's time series
 * Minter's time series
 * ...

#### Collection Time Series
This monthly partitioned table contains the most important KPIs for each collection, for each day.

In [46]:
edm = imp.reload(edm)
# edm._monatToDateList(201802)

In [41]:
# tester - part 1: first run

# edm._DML_collectionTimeSeries(pathfolder_nftkpi=cf.PATH_03DM_01tempL + 'nftkpi/art/', 
#                               pathfolder_trx=cf.PATH_02NDS_main + 'transactions/art/', 
#                               pathfolder_colldistuser=cf.PATH_03DM_01tempL + 'colldistowner/art/',
#                               pathfolder_collection_ts=cf.PATH_03DM_main + 'collection_ts/art/',
#                               actmonat=201711, malist=[7, 14, 28], _estcalc=False)

# tmp_chk = pd.read_pickle(cf.PATH_03DM_main + 'collection_ts/art/coll_ts_201711.pickle')

# print(tmp_chk.shape[0])
# tmp_chk.head(2)

In [40]:
# tester - part 1: additional months

# edm._DML_collectionTimeSeries(pathfolder_nftkpi=cf.PATH_03DM_01tempL + 'nftkpi/art/', 
#                               pathfolder_trx=cf.PATH_02NDS_main + 'transactions/art/', 
#                               pathfolder_colldistuser=cf.PATH_03DM_01tempL + 'colldistowner/art/',
#                               pathfolder_collection_ts=cf.PATH_03DM_main + 'collection_ts/art/',
#                               actmonat=201712, malist=[7, 14, 28], _estcalc=False)
# edm._DML_collectionTimeSeries(pathfolder_nftkpi=cf.PATH_03DM_01tempL + 'nftkpi/art/', 
#                               pathfolder_trx=cf.PATH_02NDS_main + 'transactions/art/', 
#                               pathfolder_colldistuser=cf.PATH_03DM_01tempL + 'colldistowner/art/',
#                               pathfolder_collection_ts=cf.PATH_03DM_main + 'collection_ts/art/',
#                               actmonat=201801, malist=[7, 14, 28], _estcalc=False)

# tmp_chk = pd.read_pickle(cf.PATH_03DM_main + 'collection_ts/art/coll_ts_201711.pickle').append(
#     pd.read_pickle(cf.PATH_03DM_main + 'collection_ts/art/coll_ts_201712.pickle')).append(
#         pd.read_pickle(cf.PATH_03DM_main + 'collection_ts/art/coll_ts_201801.pickle'))

# print(tmp_chk.shape[0])
# tmp_chk.sort_values(by=['collection_ID', 'token_category', 'report_dt'])
# tmp_chk.head(2)

In [48]:
# Run the full layer, and read the latest table
edm.collection_ts_runner(trx_folder=cf.PATH_02NDS_main + 'transactions/', 
                         nftkpi_folder=cf.PATH_03DM_01tempL + 'nftkpi/', 
                         colldistu_folder=cf.PATH_03DM_01tempL + 'colldistowner/', 
                         collts_folder=cf.PATH_03DM_main + 'collection_ts/', 
                         tokencats=['Art'], load_interval_intmonat=[], malist=[7, 14, 28], _estcalc=False, 
                         _verbose=True)

In [49]:
tmp_chk = pd.read_pickle(cf.PATH_03DM_main + 'collection_ts/art/coll_ts_202103.pickle')
tmp_chk.head(2)

,token_category,collection_ID,report_dt,coll_owner_cumdcnt,coll_token_dcnt,coll_token_sum_usd,coll_token_avg_latest_val_usd,coll_token_avg_latest_trx_day,coll_trx_maxval_usd_max,coll_trx_usd_csum_max,coll_trx_cnt_csum_max,coll_trx_cnt_dly_sum,coll_trx_usd_dly_sum,coll_avg_price_dly,coll_avg_price_dly_ma7,coll_avg_price_dly_ma14,coll_avg_price_dly_ma28,coll_trx_cnt_dly_ma7_sum,coll_trx_usd_dly_ma7_sum,coll_trx_usd_dly_ma7_min,coll_trx_usd_dly_ma7_max,coll_trx_usd_dly_ma7_std,coll_trx_usd_dly_ma7_p25,coll_trx_usd_dly_ma7_p75,coll_trx_cnt_dly_ma14_sum,coll_trx_usd_dly_ma14_sum,coll_trx_usd_dly_ma14_min,coll_trx_usd_dly_ma14_max,coll_trx_usd_dly_ma14_std,coll_trx_usd_dly_ma14_p25,coll_trx_usd_dly_ma14_p75,coll_trx_cnt_dly_ma28_sum,coll_trx_usd_dly_ma28_sum,coll_trx_usd_dly_ma28_min,coll_trx_usd_dly_ma28_max,coll_trx_usd_dly_ma28_std,coll_trx_usd_dly_ma28_p25,coll_trx_usd_dly_ma28_p75
0,Art,0xmons-xyz,2021-03-01,35.0,30.0,247568.171875,8252.272396,10.266667,19270.111328,249579.890625,32.0,31.0,248917.53125,8029.597782,11131.602539,9997.653646,8029.597782,8.0,89052.820312,7383.450195,17781.839844,3942.606209,8463.879883,12961.646484,24.0,239943.687500,5692.387695,19270.111328,3691.376580,7643.722656,11650.169922,31.0,248917.53125,725.835999,19270.111328,4926.378146,5744.994629,9920.898926
1,Art,0xmons-xyz,2021-03-02,36.0,30.0,248331.515625,8277.717188,10.933333,19270.111328,258073.718750,33.0,31.0,256685.53125,8280.178427,11567.482143,9937.500625,8280.178427,7.0,80972.375000,7383.450195,17781.839844,4041.556649,8705.011719,14109.772461,25.0,248437.515625,5692.387695,19270.111328,3626.149397,7730.479980,11595.719727,31.0,256685.53125,798.493530,19270.111328,4736.384404,5797.601562,9920.898926


In [56]:
print(tmp_chk.collection_ID.nunique())
# tmp_chk[tmp_chk.coll_avg_price_dly_ma7 < 5000].coll_avg_price_dly_ma7.plot(kind='hist')
print(tmp_chk.coll_avg_price_dly_ma7.max())
print(tmp_chk[tmp_chk.coll_avg_price_dly_ma7 == tmp_chk.coll_avg_price_dly_ma7.max()].iloc[0].collection_ID)

449
611985.5625
Beeple-special-edition


In [57]:
pdf_nft_master = pd.read_pickle(cf.PATH_02NDS_main + '/tokens/token_master_art.pickle')
pdf_nft_master.head(2)

,collection_ID,token_ID,token_URL_latest,token_permanent_link,token_mint_crypto,token_mint_price_USD,token_mint_price_crypto,token_mint_address_est,token_mint_date_est,token_latest_sales_crypto,token_latest_sales_date,token_latest_price_USD,token_latest_price_crypto,token_collection_nm,token_name,token_description,token_category,token_ID_error
0,Cryptokitties,Cryptokitties_1268119,https://lh3.googleusercontent.com/SWIgsdqj2cUc...,https://opensea.io/assets/0x06012c8cf97bead5de...,ETH,0.321473,0.001980,0xb367b96bd9af396dc5281cfdcd9e9571f670832f,2019-04-27 17:45:40,ETH,2021-04-12 17:25:55,19.363050,0.0090,Cryptokitties,I Miss You,"Guten tag. I'm I Miss You, fairy princess of C...",Art,0
1,Cryptokitties,Cryptokitties_1434540,https://lh3.googleusercontent.com/7ILL0U1wFba7...,https://opensea.io/assets/0x06012c8cf97bead5de...,ETH,0.402917,0.001487,0xfc624f8f58db41bdb95aedee1de3c1cf047105f1,2019-03-10 03:11:34,WETH,2021-04-10 11:55:04,56.558319,0.0273,Cryptokitties,Lieutenant Bippitynose,Oh heyyy. I'm Lieutenant Bippitynose. Are you ...,Art,0


In [61]:
pdf_nft_master[pdf_nft_master.token_ID == 'Beeple-special-edition_100030062'].iloc[0].token_URL_latest

'https://lh3.googleusercontent.com/79JB1V9gSMv83iMs-Db4qdpPVdpKiyt_sjGLgs575wLEgWjSRLCzMQMoxaealYz7AVCKY_mzSy6GlOYdGt_7aaCZfg=s128'

In [69]:
aaa = list(tmp_chk.collection_ID.to_list())


In [113]:
# It is still like an error...
# pdf_chk[(pdf_chk.token_ID == 'Rarible_37408') & (pdf_chk.trx_duplication_flg == 0)].sort_values(by='trx_time_sec')

In [21]:
#pdf_tokenchk = pd.read_pickle(cf.PATH_02NDS_main + 'tokens/token_master_art.pickle')
pdf_tokenchk[pdf_tokenchk.collection_ID == '0xmons-xyz'].token_mint_date_est.min() #.token_permanent_link.iloc[0]

Timestamp('2021-02-01 08:01:48')

In [72]:
date_1 = pdf_chk[pdf_chk.token_ID == 'Cryptokitties_560458'].iloc[1].trx_time_sec
date_2 = pdf_chk[pdf_chk.token_ID == 'Cryptokitties_560458'].iloc[3].trx_time_sec
float((date_2 - date_1).days)

1.0

In [94]:
edm._monat_list_creator(['201711', '201803'])

[201710, 201711, 201712, 201801, 201802, 201803]